做未来做预测

现在比较流行的方式是预测股价，我觉得不是很靠谱。

## 我想到的一个是找最低点。
这样的话，问题就由一个回归问题变为一个分类问题。  
如以过去N天的(股价，成交量，时间（季度、月份），其它股票、指数、汇率、油价，等等)组成一个特征序列，来预测次日的股价是否是最低点。
最低点的判断依据：未来T天的股价收盘价均不小于预测序列最后一日的收盘价，且未来T天的收盘价最高点大于预测序列最后一日的收盘价的Z%

In [7]:
import pandas as pd

In [1]:
import tushare as ts

ts.set_token('5fd1639100f8a22b7f86e882e03192009faa72bae1ae93803e1172d5')

pro = ts.pro_api()

In [2]:
df = pro.index_daily(ts_code='000001.SH')

In [3]:
df.head()

,ts_code,trade_date,close,open,high,low,pre_close,change,pct_chg,vol,amount
0,000001.SH,20181128,2601.7365,2575.4541,2601.9626,2561.5618,2574.6792,27.0573,1.0509,145963473.0,119578491.7
1,000001.SH,20181127,2574.6792,2585.8261,2592.5353,2566.1663,2575.8101,-1.1309,-0.0439,123658436.0,102226561.4
2,000001.SH,20181126,2575.8101,2580.8424,2594.9966,2568.0352,2579.4831,-3.6730,-0.1424,134314540.0,108158093.0
3,000001.SH,20181123,2579.4831,2640.6674,2642.0356,2577.3511,2645.4339,-65.9508,-2.4930,191474549.0,149975643.9
4,000001.SH,20181122,2645.4339,2655.8964,2658.0011,2634.4827,2651.5052,-6.0713,-0.2290,149309063.0,125570173.8


In [4]:
df.tail()

,ts_code,trade_date,close,open,high,low,pre_close,change,pct_chg,vol,amount
1795,000001.SH,20110713,2795.476,2756.032,2797.266,2756.032,2754.582,40.894,1.4846,91929128.0,1.076960e+08
1796,000001.SH,20110712,2754.582,2783.722,2783.722,2752.186,2802.692,-48.110,-1.7166,96674475.0,1.115352e+08
1797,000001.SH,20110711,2802.692,2789.454,2807.322,2780.665,2797.774,4.918,0.1758,86063595.0,1.014421e+08
1798,000001.SH,20110708,2797.774,2797.026,2807.380,2784.126,2794.267,3.507,0.1255,86211609.0,1.027651e+08
1799,000001.SH,20110707,2794.267,2813.193,2825.123,2793.892,2810.479,-16.212,-0.5768,116512143.0,1.400651e+08


In [5]:
dff = pro.index_daily(ts_code='000001.SH', start_date='20040216', end_date='20110706')

In [8]:
dffa = pd.concat([df,dff], axis=0)

In [9]:
df1 = dffa.loc[:,['ts_code', 'trade_date', 'close', 'vol', 'amount']]
df1.head()

,ts_code,trade_date,close,vol,amount
0,000001.SH,20181128,2601.7365,145963473.0,119578491.7
1,000001.SH,20181127,2574.6792,123658436.0,102226561.4
2,000001.SH,20181126,2575.8101,134314540.0,108158093.0
3,000001.SH,20181123,2579.4831,191474549.0,149975643.9
4,000001.SH,20181122,2645.4339,149309063.0,125570173.8


In [10]:
def normlize_field(df, fieldnamelist):
    '''
    #设df中某一字段 第一日净值为1
    '''
    for f in fieldnamelist:

        df[f+'_1'] = df.apply(lambda x: x[f]/df1.iloc[df.shape[0]-1][f], axis=1)

In [11]:
normlize_field(df1, ['close','vol','amount'])

In [12]:
df1.head()

,ts_code,trade_date,close,vol,amount,close_1,vol_1,amount_1
0,000001.SH,20181128,2601.7365,145963473.0,119578491.7,1.533295,6.348693,6.157281
1,000001.SH,20181127,2574.6792,123658436.0,102226561.4,1.517349,5.378534,5.263804
2,000001.SH,20181126,2575.8101,134314540.0,108158093.0,1.518016,5.842022,5.569227
3,000001.SH,20181123,2579.4831,191474549.0,149975643.9,1.520180,8.328201,7.722478
4,000001.SH,20181122,2645.4339,149309063.0,125570173.8,1.559048,6.494210,6.465802


In [13]:
df2 = df1.sort_values('trade_date', ascending=True)
df2.head()

,ts_code,trade_date,close,vol,amount,close_1,vol_1,amount_1
1799,000001.SH,20040216,1696.827,22991106.0,1.942066e+07,1.000000,1.000000,1.000000
1798,000001.SH,20040217,1703.774,29754209.0,2.588424e+07,1.004094,1.294162,1.332819
1797,000001.SH,20040218,1717.096,29218375.0,2.548816e+07,1.011945,1.270856,1.312425
1796,000001.SH,20040219,1717.015,27338781.0,2.364013e+07,1.011898,1.189102,1.217267
1795,000001.SH,20040220,1721.752,23409159.0,1.995414e+07,1.014689,1.018183,1.027470


In [14]:
def build_flag(df,series_len,pro_len):
    '''
    构建训练集
    
    series_len: 参考的之前的序列范围。如以之前的series_len个序列预测下一个序列，则series_len=series_len
    pro_len: 预测日以后的天数（含预测日）
    '''
    ll = list()
    l1 = list(df['close_1'])
    l4 = list(df['vol_1'])
    l5 = list(df['amount_1'])
    for i in range(series_len,(df.shape[0]-pro_len)):
        final_list = list()
        l2 = l1[i-series_len : i]
        l3 = l1[i:i+pro_len]
        #最低点和买点的关系：最低点一定是买点，买点不一定是最低点
        #买点特征
        #买点日之后14日最高收盘价涨幅超过0.05，最低价不得低于买点日收盘价
        #最低点的特征：在买点特征的基础上
        #最低日收盘价低于前一日收盘价

        f1 = 0 # 买点标志 1表示买点
        f2 = 0 # 最低点标志 1表示最低点
        if (max(l3) - l1[i])/l1[i] > 0.05 and min(l3[1:]) > l1[i]:#未来pro_len日最高收盘价涨幅超过0.05
            f1 = 1
            if l1[i] < l1[i-1]:
                f2 = 1
        final_list.append(list(zip(l1[i-series_len : i],l4[i-series_len : i],l5[i-series_len : i])))
        final_list.append(f1)
        final_list.append(f2)
        ll.append(final_list)    
    return ll

In [65]:
ll = build_flag(df2,180,15)
ll[:5]

[[[(1.0, 1.0, 1.0),
   (1.0040941121281073, 1.2941617075750944, 1.332819407903278),
   (1.0119452366092714, 1.2708555647562148, 1.3124245934986103),
   (1.011897500452315, 1.1891024729301845, 1.2172667282899259),
   (1.0146891816313626, 1.018183248774548, 1.0274695841628634),
   (0.9910580159320898, 1.2423488891747965, 1.2299252368464704),
   (0.9914369585113862, 1.1743402427008078, 1.1305174547033425),
   (0.9708597281867863, 1.0693789589765712, 1.041228335492268),
   (0.9752897614194022, 1.0052445497837295, 0.9405989823856901),
   (0.9871760645015668, 0.8572168298471592, 0.8476502332207176),
   (0.9958351676393645, 0.8694743958816075, 0.8801622380218815),
   (0.9943724374965744, 0.8919754882605474, 0.8952930459535767),
   (0.9793644254835643, 0.8368442127142557, 0.8213201835875515),
   (0.9835263111678445, 0.777928169266846, 0.7572829778961754),
   (0.9795317967005476, 0.6992081633654336, 0.6828425689988216),
   (0.9648267030168662, 0.6802276062752266, 0.6611068173195006),
   (0.9648

In [66]:
y = [(i[-2],i[-1]) for i in ll]

In [67]:
len(y)

3405

In [68]:
from collections import Counter
Counter(y)

Counter({(1, 1): 184, (1, 0): 242, (0, 0): 2979})

In [73]:
Counter(y[:3000])

Counter({(1, 1): 179, (1, 0): 235, (0, 0): 2586})

In [74]:
Counter(y[3000:])

Counter({(0, 0): 393, (1, 1): 5, (1, 0): 7})

In [76]:
df3 = df2.iloc[180:(df2.shape[0]-15)]
df3['y1'] = [i[0] for i in y]
df3['y2'] = [i[1] for i in y]

/home/ian/installed/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/ian/installed/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [77]:
df3

,ts_code,trade_date,close,vol,amount,close_1,vol_1,amount_1,y1,y2
1619,000001.SH,20041108,1304.2290,7376656.0,4.398204e+06,0.768628,0.320848,0.226470,1,1
1618,000001.SH,20041109,1307.4290,7111640.0,4.270396e+06,0.770514,0.309321,0.219889,1,0
1617,000001.SH,20041110,1354.3860,23319738.0,1.410214e+07,0.798187,1.014294,0.726141,0,0
1616,000001.SH,20041111,1347.0700,27429895.0,1.680318e+07,0.793876,1.193065,0.865221,0,0
1615,000001.SH,20041112,1352.2170,15744105.0,9.490678e+06,0.796909,0.684791,0.488690,0,0
1614,000001.SH,20041115,1370.0460,14230535.0,8.929426e+06,0.807416,0.618958,0.459790,0,0
1613,000001.SH,20041116,1370.3890,14165607.0,8.616093e+06,0.807619,0.616134,0.443656,0,0
1612,000001.SH,20041117,1356.1380,12234261.0,7.593393e+06,0.799220,0.532130,0.390996,0,0
1611,000001.SH,20041118,1367.8280,11589095.0,7.315559e+06,0.806109,0.504069,0.376689,0,0
1610,000001.SH,20041119,1379.9600,14107904.0,8.670277e+06,0.813259,0.613624,0.446446,0,0


## 数据预处理

In [78]:
x = np.array([i[0] for i in ll])
y1 = np.array([[i[1]] for i in ll])#买点
y2 = np.array([[i[2]] for i in ll])#最低点

x.shape, y1.shape, y2.shape

((3405, 180, 3), (3405, 1), (3405, 1))

In [79]:
x1 = x.reshape(x.shape[0],-1)
x_train, x_test = x1[:3200], x1[3200:]

In [80]:
x1.shape

(3405, 540)

In [81]:
y11 = np_utils.to_categorical(y1,num_classes=2)
y_train, y_test = y11[:3200], y11[3200:]
y11.shape

(3405, 2)

## 利用BP做预测

In [82]:
import numpy as np
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD

In [144]:
np.array([[i[1]] for i in ll])

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [157]:
# 创建模型，输入784个神经元，输出10个神经元
model = Sequential()
model.add(Dense(units=100,input_dim=90,bias_initializer='one',activation='tanh'))
model.add(Dense(units=500,bias_initializer='one',activation='tanh'))
model.add(Dense(units=500,bias_initializer='one',activation='tanh'))
model.add(Dense(units=2,activation='tanh'))

# 定义优化器
sgd = SGD(lr=0.2)

# 定义优化器，loss function，训练过程中计算准确率
model.compile(
    optimizer = sgd,
    loss = 'mse',
    metrics=['accuracy'],
)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             (None, 100)               9100      
_________________________________________________________________
dense_29 (Dense)             (None, 500)               50500     
_________________________________________________________________
dense_30 (Dense)             (None, 500)               250500    
_________________________________________________________________
dense_31 (Dense)             (None, 2)                 1002      
Total params: 311,102
Trainable params: 311,102
Non-trainable params: 0
_________________________________________________________________


In [158]:
# 训练模型
model.fit(x_train,y_train,batch_size=128,epochs=100)

# 评估模型
loss,accuracy = model.evaluate(x_test,y_test)

print('\ntest loss',loss)
print('accuracy',accuracy)

Epoch 1/100
3200/3200 [==============================] - 0s 132us/step - loss: 0.5184 - acc: 0.8687
Epoch 2/100
3200/3200 [==============================] - 0s 49us/step - loss: 0.5000 - acc: 0.8687
Epoch 3/100
3200/3200 [==============================] - 0s 46us/step - loss: 0.5000 - acc: 0.8687
Epoch 4/100
3200/3200 [==============================] - 0s 48us/step - loss: 0.5000 - acc: 0.8687
Epoch 5/100
3200/3200 [==============================] - 0s 40us/step - loss: 0.5000 - acc: 0.8687
Epoch 6/100
3200/3200 [==============================] - 0s 47us/step - loss: 0.5000 - acc: 0.8687
Epoch 7/100
3200/3200 [==============================] - 0s 47us/step - loss: 0.5000 - acc: 0.8687
Epoch 8/100
3200/3200 [==============================] - 0s 48us/step - loss: 0.5000 - acc: 0.8687
Epoch 9/100
3200/3200 [==============================] - 0s 49us/step - loss: 0.5000 - acc: 0.8687
Epoch 10/100
3200/3200 [==============================] - 0s 48us/step - loss: 0.5000 - acc: 0.8687
Epoch 11

3200/3200 [==============================] - 0s 47us/step - loss: 0.5000 - acc: 0.8687
Epoch 84/100
3200/3200 [==============================] - 0s 51us/step - loss: 0.5000 - acc: 0.8687
Epoch 85/100
3200/3200 [==============================] - 0s 48us/step - loss: 0.5000 - acc: 0.8687
Epoch 86/100
3200/3200 [==============================] - 0s 46us/step - loss: 0.5000 - acc: 0.8687
Epoch 87/100
3200/3200 [==============================] - 0s 46us/step - loss: 0.5000 - acc: 0.8687
Epoch 88/100
3200/3200 [==============================] - 0s 48us/step - loss: 0.5000 - acc: 0.8687
Epoch 89/100
3200/3200 [==============================] - 0s 43us/step - loss: 0.5000 - acc: 0.8687
Epoch 90/100
3200/3200 [==============================] - 0s 44us/step - loss: 0.5000 - acc: 0.8687
Epoch 91/100
3200/3200 [==============================] - 0s 47us/step - loss: 0.5000 - acc: 0.8687
Epoch 92/100
3200/3200 [==============================] - 0s 49us/step - loss: 0.5000 - acc: 0.8687
Epoch 93/100


## 使用CNN

In [28]:
from keras.layers import Dense,Dropout,Convolution2D,MaxPooling2D,Flatten
from keras.optimizers import Adam
import time

In [29]:
# 定义顺序模型
model = Sequential()

# 第一个卷积层
# input_shape 输入平面
# filters 卷积核/滤波器个数
# kernel_size 卷积窗口大小
# strides 步长
# padding padding方式 same/valid
# activation 激活函数
model.add(Convolution2D(
    input_shape = (10,9,1),
    filters = 64,
    kernel_size = 5,
    strides = 1,
    padding = 'same',
    activation = 'relu'
))
# 第一个池化层
model.add(MaxPooling2D(
    pool_size = 2,
    strides = 2,
    padding = 'same',
))
# 第二个卷积层
model.add(Convolution2D(126,5,strides=1,padding='same',activation = 'relu'))
# 第二个池化层
model.add(MaxPooling2D(2,2,'same'))

# 第三个卷积层
model.add(Convolution2D(256,5,strides=1,padding='same',activation = 'relu'))
# 第三个池化层
model.add(MaxPooling2D(2,2,'same'))

# 把第三个池化层的输出扁平化为1维
model.add(Flatten())
# 第一个全连接层
model.add(Dense(1024,activation = 'relu'))
# Dropout
model.add(Dropout(0.5))
# 第二个全连接层
model.add(Dense(2,activation='softmax'))

# 定义优化器
adam = Adam(lr=1e-4)

# 定义优化器，loss function，训练过程中计算准确率
model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 10, 9, 64)         1664      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 5, 5, 126)         201726    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 3, 126)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 256)         806656    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 2, 2, 256)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1024)              0         
__________

In [30]:
start = time.time()
# 训练模型
model.fit(x_train.reshape((x_train.shape[0],10,9,1)),y_train,batch_size=256,epochs=266)
print('@ Total Time Spent: %.2f seconds' % (time.time() - start))

Epoch 1/266
3200/3200 [==============================] - 2s 739us/step - loss: 0.5231 - acc: 0.8419
Epoch 2/266
3200/3200 [==============================] - 2s 534us/step - loss: 0.4764 - acc: 0.8687
Epoch 3/266
3200/3200 [==============================] - 2s 588us/step - loss: 0.4567 - acc: 0.8687
Epoch 4/266
3200/3200 [==============================] - 2s 545us/step - loss: 0.4319 - acc: 0.8694
Epoch 5/266
3200/3200 [==============================] - 2s 539us/step - loss: 0.4317 - acc: 0.8663
Epoch 6/266
3200/3200 [==============================] - 2s 545us/step - loss: 0.4137 - acc: 0.8684
Epoch 7/266
3200/3200 [==============================] - 2s 529us/step - loss: 0.4128 - acc: 0.8675
Epoch 8/266
3200/3200 [==============================] - 2s 522us/step - loss: 0.4165 - acc: 0.8678
Epoch 9/266
3200/3200 [==============================] - 2s 543us/step - loss: 0.4117 - acc: 0.8669
Epoch 10/266
3200/3200 [==============================] - 2s 527us/step - loss: 0.4056 - acc: 0.8687

3200/3200 [==============================] - 2s 532us/step - loss: 0.1221 - acc: 0.9550
Epoch 83/266
3200/3200 [==============================] - 2s 516us/step - loss: 0.1384 - acc: 0.9484
Epoch 84/266
3200/3200 [==============================] - 2s 535us/step - loss: 0.1285 - acc: 0.9528
Epoch 85/266
3200/3200 [==============================] - 2s 526us/step - loss: 0.1586 - acc: 0.9391
Epoch 86/266
3200/3200 [==============================] - 2s 533us/step - loss: 0.1285 - acc: 0.9522
Epoch 87/266
3200/3200 [==============================] - 2s 534us/step - loss: 0.1350 - acc: 0.9441
Epoch 88/266
3200/3200 [==============================] - 2s 531us/step - loss: 0.1317 - acc: 0.9528
Epoch 89/266
3200/3200 [==============================] - 2s 534us/step - loss: 0.1188 - acc: 0.9569
Epoch 90/266
3200/3200 [==============================] - 2s 524us/step - loss: 0.1049 - acc: 0.9631
Epoch 91/266
3200/3200 [==============================] - 2s 531us/step - loss: 0.1142 - acc: 0.9581
Epo

3200/3200 [==============================] - 2s 544us/step - loss: 0.0197 - acc: 0.9969
Epoch 163/266
3200/3200 [==============================] - 2s 544us/step - loss: 0.0221 - acc: 0.9966
Epoch 164/266
3200/3200 [==============================] - 2s 540us/step - loss: 0.0205 - acc: 0.9966
Epoch 165/266
3200/3200 [==============================] - 2s 544us/step - loss: 0.0184 - acc: 0.9972
Epoch 166/266
3200/3200 [==============================] - 2s 547us/step - loss: 0.0168 - acc: 0.9969
Epoch 167/266
3200/3200 [==============================] - 2s 527us/step - loss: 0.0171 - acc: 0.9975
Epoch 168/266
3200/3200 [==============================] - 2s 514us/step - loss: 0.0161 - acc: 0.9981
Epoch 169/266
3200/3200 [==============================] - 2s 533us/step - loss: 0.0163 - acc: 0.9969
Epoch 170/266
3200/3200 [==============================] - 2s 555us/step - loss: 0.0162 - acc: 0.9975
Epoch 171/266
3200/3200 [==============================] - 2s 525us/step - loss: 0.0161 - acc: 0

3200/3200 [==============================] - 2s 527us/step - loss: 0.1391 - acc: 0.9519
Epoch 243/266
3200/3200 [==============================] - 2s 539us/step - loss: 0.1439 - acc: 0.9409
Epoch 244/266
3200/3200 [==============================] - 2s 514us/step - loss: 0.0880 - acc: 0.9731
Epoch 245/266
3200/3200 [==============================] - 2s 530us/step - loss: 0.0376 - acc: 0.9875
Epoch 246/266
3200/3200 [==============================] - 2s 524us/step - loss: 0.0225 - acc: 0.9969
Epoch 247/266
3200/3200 [==============================] - 2s 544us/step - loss: 0.0236 - acc: 0.9953
Epoch 248/266
3200/3200 [==============================] - 2s 535us/step - loss: 0.0189 - acc: 0.9972
Epoch 249/266
3200/3200 [==============================] - 2s 536us/step - loss: 0.0133 - acc: 0.9991
Epoch 250/266
3200/3200 [==============================] - 2s 544us/step - loss: 0.0105 - acc: 0.9997
Epoch 251/266
3200/3200 [==============================] - 2s 526us/step - loss: 0.0102 - acc: 0

In [31]:
p = model.predict(x_test.reshape((x_test.shape[0],10,9,1)))

In [32]:
p[-10:]

array([[9.9998593e-01, 1.4124693e-05],
       [1.0000000e+00, 4.3917570e-10],
       [1.0000000e+00, 8.7278396e-10],
       [9.9999309e-01, 6.8770369e-06],
       [9.9999988e-01, 7.6666971e-08],
       [1.0000000e+00, 5.8134241e-08],
       [9.9999738e-01, 2.6630350e-06],
       [9.9998093e-01, 1.9049217e-05],
       [1.0000000e+00, 2.8458558e-08],
       [9.9926537e-01, 7.3465967e-04]], dtype=float32)

In [33]:
model.evaluate(x_test.reshape((x_test.shape[0],10,9,1)),y_test)

355/355 [==============================] - 0s 386us/step


[0.582860738884114, 0.9492957746478873]

In [168]:
y1[-10:]

array([[0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0]])

## 使用SimpleRNN预测

In [34]:
from keras.layers.recurrent import SimpleRNN

In [36]:
# 创建模型
model = Sequential()

# 循环神经网络
model.add(SimpleRNN(
    units = 50, # 输出
    input_shape = (30,3), #输入
))

# 输出层
model.add(Dense(2,activation='softmax'))

# 定义优化器
adam = Adam(lr=1e-4)

# 定义优化器，loss function，训练过程中计算准确率
model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 50)                2700      
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 102       
Total params: 2,802
Trainable params: 2,802
Non-trainable params: 0
_________________________________________________________________


In [38]:
# 训练模型
model.fit(x[:3200],y_train[:3200],batch_size=64,epochs=10)

Epoch 1/10
3200/3200 [==============================] - 1s 263us/step - loss: 0.4818 - acc: 0.8672
Epoch 2/10
3200/3200 [==============================] - 1s 175us/step - loss: 0.4001 - acc: 0.8688
Epoch 3/10
3200/3200 [==============================] - 1s 178us/step - loss: 0.3906 - acc: 0.8688
Epoch 4/10
3200/3200 [==============================] - 1s 174us/step - loss: 0.3893 - acc: 0.8688
Epoch 5/10
3200/3200 [==============================] - 1s 173us/step - loss: 0.3888 - acc: 0.8688
Epoch 6/10
3200/3200 [==============================] - 1s 176us/step - loss: 0.3884 - acc: 0.8688
Epoch 7/10
3200/3200 [==============================] - 1s 164us/step - loss: 0.3878 - acc: 0.8688
Epoch 8/10
3200/3200 [==============================] - 1s 183us/step - loss: 0.3874 - acc: 0.8688
Epoch 9/10
3200/3200 [==============================] - 1s 176us/step - loss: 0.3869 - acc: 0.8688
Epoch 10/10
3200/3200 [==============================] - 1s 175us/step - loss: 0.3864 - acc: 0.8688


In [41]:
# 训练模型
model.fit(x[:3200],y_train[:3200],batch_size=64,epochs=200)

Epoch 1/200
3200/3200 [==============================] - 0s 150us/step - loss: 0.3267 - acc: 0.8759
Epoch 2/200
3200/3200 [==============================] - 0s 155us/step - loss: 0.3344 - acc: 0.8709
Epoch 3/200
3200/3200 [==============================] - 1s 176us/step - loss: 0.3281 - acc: 0.8744
Epoch 4/200
3200/3200 [==============================] - 1s 163us/step - loss: 0.3265 - acc: 0.8744
Epoch 5/200
3200/3200 [==============================] - 1s 161us/step - loss: 0.3263 - acc: 0.8747
Epoch 6/200
3200/3200 [==============================] - 1s 171us/step - loss: 0.3314 - acc: 0.8728
Epoch 7/200
3200/3200 [==============================] - 0s 151us/step - loss: 0.3254 - acc: 0.8769
Epoch 8/200
3200/3200 [==============================] - 1s 168us/step - loss: 0.3251 - acc: 0.8759
Epoch 9/200
3200/3200 [==============================] - 1s 169us/step - loss: 0.3254 - acc: 0.8769
Epoch 10/200
3200/3200 [==============================] - 1s 180us/step - loss: 0.3298 - acc: 0.8762

3200/3200 [==============================] - 1s 164us/step - loss: 0.3234 - acc: 0.8738
Epoch 83/200
3200/3200 [==============================] - 0s 155us/step - loss: 0.3224 - acc: 0.8756
Epoch 84/200
3200/3200 [==============================] - 0s 149us/step - loss: 0.3280 - acc: 0.8738
Epoch 85/200
3200/3200 [==============================] - 0s 155us/step - loss: 0.3217 - acc: 0.8784
Epoch 86/200
3200/3200 [==============================] - 0s 153us/step - loss: 0.3245 - acc: 0.8775
Epoch 87/200
3200/3200 [==============================] - 0s 150us/step - loss: 0.3239 - acc: 0.8772
Epoch 88/200
3200/3200 [==============================] - 0s 151us/step - loss: 0.3207 - acc: 0.8766
Epoch 89/200
3200/3200 [==============================] - 1s 164us/step - loss: 0.3215 - acc: 0.8794
Epoch 90/200
3200/3200 [==============================] - 0s 152us/step - loss: 0.3229 - acc: 0.8784
Epoch 91/200
3200/3200 [==============================] - 1s 177us/step - loss: 0.3239 - acc: 0.8778
Epo

3200/3200 [==============================] - 0s 153us/step - loss: 0.3223 - acc: 0.8803
Epoch 163/200
3200/3200 [==============================] - 1s 170us/step - loss: 0.3325 - acc: 0.8694
Epoch 164/200
3200/3200 [==============================] - 1s 158us/step - loss: 0.3228 - acc: 0.8778
Epoch 165/200
3200/3200 [==============================] - 0s 146us/step - loss: 0.3192 - acc: 0.8787
Epoch 166/200
3200/3200 [==============================] - 1s 191us/step - loss: 0.3177 - acc: 0.8784
Epoch 167/200
3200/3200 [==============================] - 1s 167us/step - loss: 0.3185 - acc: 0.8781
Epoch 168/200
3200/3200 [==============================] - 0s 154us/step - loss: 0.3177 - acc: 0.8791
Epoch 169/200
3200/3200 [==============================] - 1s 157us/step - loss: 0.3210 - acc: 0.8791
Epoch 170/200
3200/3200 [==============================] - 0s 155us/step - loss: 0.3232 - acc: 0.8778
Epoch 171/200
3200/3200 [==============================] - 1s 161us/step - loss: 0.3222 - acc: 0

In [42]:
# 评估模型
loss,accuracy = model.evaluate(x[3200:],y_test)

print('test loss',loss)
print('test accuracy',accuracy)

p = model.predict(x[3200:])

p[-10:]

355/355 [==============================] - 0s 337us/step
test loss 0.2047892114226247
test accuracy 0.9183098591549296


## LSTM

In [83]:
from keras.layers.recurrent import LSTM

In [86]:
# 创建模型
model = Sequential()

# 循环神经网络
model.add(LSTM(
    units = 256, # 输出
    input_shape = (180,3), #输入
))
model.add(Dense(200,activation='tanh'))
# 输出层
model.add(Dense(2,activation='softmax'))

# 定义优化器
adam = Adam(lr=1e-4)

# 定义优化器，loss function，训练过程中计算准确率
model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 256)               266240    
_________________________________________________________________
dense_11 (Dense)             (None, 200)               51400     
_________________________________________________________________
dense_12 (Dense)             (None, 2)                 402       
Total params: 318,042
Trainable params: 318,042
Non-trainable params: 0
_________________________________________________________________


In [87]:
# 训练模型
model.fit(x[:3000],y_train[:3000],batch_size=128,epochs=300)

Epoch 1/300
3000/3000 [==============================] - 17s 6ms/step - loss: 0.5505 - acc: 0.7250
Epoch 2/300
3000/3000 [==============================] - 16s 5ms/step - loss: 0.4307 - acc: 0.8620
Epoch 3/300
3000/3000 [==============================] - 15s 5ms/step - loss: 0.4145 - acc: 0.8620
Epoch 4/300
3000/3000 [==============================] - 15s 5ms/step - loss: 0.4102 - acc: 0.8620
Epoch 5/300
3000/3000 [==============================] - 16s 5ms/step - loss: 0.4057 - acc: 0.8620
Epoch 6/300
3000/3000 [==============================] - 16s 5ms/step - loss: 0.4021 - acc: 0.8620
Epoch 7/300
3000/3000 [==============================] - 16s 5ms/step - loss: 0.3996 - acc: 0.8620
Epoch 8/300
3000/3000 [==============================] - 15s 5ms/step - loss: 0.3972 - acc: 0.8620
Epoch 9/300
3000/3000 [==============================] - 15s 5ms/step - loss: 0.3955 - acc: 0.8620
Epoch 10/300
3000/3000 [==============================] - 16s 5ms/step - loss: 0.3932 - acc: 0.8620
Epoch 11/

3000/3000 [==============================] - 16s 5ms/step - loss: 0.3177 - acc: 0.8747
Epoch 84/300
3000/3000 [==============================] - 15s 5ms/step - loss: 0.3107 - acc: 0.8753
Epoch 85/300
3000/3000 [==============================] - 16s 5ms/step - loss: 0.3103 - acc: 0.8710
Epoch 86/300
3000/3000 [==============================] - 16s 5ms/step - loss: 0.3100 - acc: 0.8730
Epoch 87/300
3000/3000 [==============================] - 15s 5ms/step - loss: 0.3143 - acc: 0.8730
Epoch 88/300
3000/3000 [==============================] - 16s 5ms/step - loss: 0.3212 - acc: 0.8693
Epoch 89/300
3000/3000 [==============================] - 15s 5ms/step - loss: 0.3398 - acc: 0.8610
Epoch 90/300
3000/3000 [==============================] - 15s 5ms/step - loss: 0.3390 - acc: 0.8670
Epoch 91/300
3000/3000 [==============================] - 15s 5ms/step - loss: 0.3566 - acc: 0.8627
Epoch 92/300
3000/3000 [==============================] - 15s 5ms/step - loss: 0.3418 - acc: 0.8707
Epoch 93/300


3000/3000 [==============================] - 15s 5ms/step - loss: 0.2593 - acc: 0.8930
Epoch 165/300
3000/3000 [==============================] - 15s 5ms/step - loss: 0.2566 - acc: 0.8897
Epoch 166/300
3000/3000 [==============================] - 15s 5ms/step - loss: 0.2516 - acc: 0.8897
Epoch 167/300
3000/3000 [==============================] - 15s 5ms/step - loss: 0.2464 - acc: 0.8940
Epoch 168/300
3000/3000 [==============================] - 16s 5ms/step - loss: 0.2489 - acc: 0.8867
Epoch 169/300
3000/3000 [==============================] - 15s 5ms/step - loss: 0.2503 - acc: 0.8910
Epoch 170/300
3000/3000 [==============================] - 15s 5ms/step - loss: 0.2621 - acc: 0.8870
Epoch 171/300
3000/3000 [==============================] - 15s 5ms/step - loss: 0.2522 - acc: 0.8930
Epoch 172/300
3000/3000 [==============================] - 15s 5ms/step - loss: 0.2531 - acc: 0.8907
Epoch 173/300
3000/3000 [==============================] - 15s 5ms/step - loss: 0.2470 - acc: 0.8923
Epoc

3000/3000 [==============================] - 15s 5ms/step - loss: 0.2293 - acc: 0.8993
Epoch 246/300
3000/3000 [==============================] - 15s 5ms/step - loss: 0.2323 - acc: 0.9003
Epoch 247/300
3000/3000 [==============================] - 15s 5ms/step - loss: 0.2233 - acc: 0.9003
Epoch 248/300
3000/3000 [==============================] - 15s 5ms/step - loss: 0.2194 - acc: 0.9057
Epoch 249/300
3000/3000 [==============================] - 15s 5ms/step - loss: 0.2221 - acc: 0.9013
Epoch 250/300
3000/3000 [==============================] - 16s 5ms/step - loss: 0.2169 - acc: 0.8990
Epoch 251/300
3000/3000 [==============================] - 15s 5ms/step - loss: 0.2133 - acc: 0.9040
Epoch 252/300
3000/3000 [==============================] - 15s 5ms/step - loss: 0.2185 - acc: 0.9000
Epoch 253/300
3000/3000 [==============================] - 15s 5ms/step - loss: 0.2176 - acc: 0.8997
Epoch 254/300
3000/3000 [==============================] - 16s 5ms/step - loss: 0.2182 - acc: 0.9013
Epoc

In [89]:
# 评估模型
loss,accuracy = model.evaluate(x[3000:],y11[3000:])

print('test loss',loss)
print('test accuracy',accuracy)

p = model.predict(x[3000:])

p[-10:]

405/405 [==============================] - 1s 3ms/step
test loss 0.30328865814192313
test accuracy 0.9703703708118863


array([[9.9999690e-01, 3.0603428e-06],
       [9.9999666e-01, 3.3833219e-06],
       [9.9999499e-01, 4.9805617e-06],
       [9.9998999e-01, 9.9705348e-06],
       [9.9999225e-01, 7.7104933e-06],
       [9.9999189e-01, 8.0543050e-06],
       [9.9999452e-01, 5.5148798e-06],
       [9.9999678e-01, 3.2015703e-06],
       [9.9999368e-01, 6.3325760e-06],
       [9.9998081e-01, 1.9225445e-05]], dtype=float32)

In [90]:
np.argmax(p, axis=1)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [91]:
np.argmax(y_test,axis=1)

array([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0])